In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from sklearn.impute import SimpleImputer
import time

In [2]:
train_data = pd.read_csv("datasets/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("datasets/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [37]:
y = train_data["Survived"]

features = ["Pclass", "Sex", "Age", "SibSp", "Parch"]

X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

my_imputer = SimpleImputer()
imputed_X = pd.DataFrame(my_imputer.fit_transform(X))
imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))

# Imputation removed column names; put them back
imputed_X.columns = X.columns
imputed_X_test.coloumns = X_test.columns

y = tf.constant(np.array([y]).T)
X = tf.constant(np.array(imputed_X))

X_test = tf.constant(np.array(imputed_X_test))

/tmp/ipykernel_12096/3955001062.py:14: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  imputed_X_test.coloumns = X_test.columns


In [38]:
X[:2]

<tf.Tensor: shape=(2, 6), dtype=float64, numpy=
array([[ 3., 22.,  1.,  0.,  0.,  1.],
       [ 1., 38.,  1.,  0.,  1.,  0.]])>

In [39]:
X = tf.keras.utils.normalize(X)
X_test = tf.keras.utils.normalize(X_test)

y.shape

TensorShape([891, 1])

In [19]:
def build_model(layers, nodes):
    NAME = f"{nodes}-nodes-{layers}-layers-{int(time.time())}"
    print(NAME)
    tensorboard = TensorBoard(log_dir=f"logs/{NAME}")
    model = tf.keras.models.Sequential()
    
    for l in range(layers):
        model.add(tf.keras.layers.Dense(nodes, activation="relu"))

    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.binary_crossentropy, metrics=["accuracy"])

    model.fit(X, y, epochs=100, batch_size=64, validation_split=0.2, verbose=1, callbacks=[tensorboard])

    return model

In [22]:
nodes = [4, 8, 12, 16, 32]

for layer in range(15):
    for node in nodes:
        model = build_model(layer, node)

4-nodes-0-layers-1635085810
Epoch 1/100


2021-10-24 20:00:10.443456: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-24 20:00:10.443501: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-24 20:00:10.444842: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


12/12 [==============================] - 0s 14ms/step - loss: 0.6726 - accuracy: 0.6096 - val_loss: 0.6701 - val_accuracy: 0.6425
Epoch 2/100
12/12 [==============================] - 0s 3ms/step - loss: 0.6715 - accuracy: 0.6096 - val_loss: 0.6685 - val_accuracy: 0.6425
Epoch 3/100
 1/12 [=>............................] - ETA: 0s - loss: 0.6576 - accuracy: 0.6562

2021-10-24 20:00:10.759594: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-24 20:00:10.759625: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-24 20:00:10.784352: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-10-24 20:00:10.785216: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-10-24 20:00:10.786478: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/4-nodes-0-layers-1635085810/train/plugins/profile/2021_10_24_20_00_10

2021-10-24 20:00:10.787197: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/4-nodes-0-layers-1635085810/train/plugins/profile/2021_10_24_20_00_10/root.trace.json.gz
2021-10-24 20:00:10.788315: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/4-nodes-0-layers-1635085

12/12 [==============================] - 0s 3ms/step - loss: 0.6705 - accuracy: 0.6096 - val_loss: 0.6673 - val_accuracy: 0.6425
Epoch 4/100
12/12 [==============================] - 0s 3ms/step - loss: 0.6697 - accuracy: 0.6096 - val_loss: 0.6660 - val_accuracy: 0.6425
Epoch 5/100
12/12 [==============================] - 0s 3ms/step - loss: 0.6690 - accuracy: 0.6096 - val_loss: 0.6652 - val_accuracy: 0.6425
Epoch 6/100
12/12 [==============================] - 0s 3ms/step - loss: 0.6685 - accuracy: 0.6096 - val_loss: 0.6643 - val_accuracy: 0.6425
Epoch 7/100
12/12 [==============================] - 0s 3ms/step - loss: 0.6680 - accuracy: 0.6096 - val_loss: 0.6637 - val_accuracy: 0.6425
Epoch 8/100
12/12 [==============================] - 0s 4ms/step - loss: 0.6675 - accuracy: 0.6096 - val_loss: 0.6627 - val_accuracy: 0.6425
Epoch 9/100
12/12 [==============================] - 0s 3ms/step - loss: 0.6669 - accuracy: 0.6096 - val_loss: 0.6616 - val_accuracy: 0.6425
Epoch 10/100
12/12 [=====

...

12/12 [==============================] - 1s 38ms/step - loss: 0.6923 - accuracy: 0.6039 - val_loss: 0.6901 - val_accuracy: 0.6425
Epoch 2/100
12/12 [==============================] - 0s 5ms/step - loss: 0.6897 - accuracy: 0.6096 - val_loss: 0.6865 - val_accuracy: 0.6425
Epoch 3/100
12/12 [==============================] - 0s 5ms/step - loss: 0.6865 - accuracy: 0.6096 - val_loss: 0.6813 - val_accuracy: 0.6425
Epoch 4/100
12/12 [==============================] - 0s 5ms/step - loss: 0.6822 - accuracy: 0.6096 - val_loss: 0.6749 - val_accuracy: 0.6425
Epoch 5/100
12/12 [==============================] - 0s 5ms/step - loss: 0.6773 - accuracy: 0.6096 - val_loss: 0.6643 - val_accuracy: 0.6425
Epoch 6/100
12/12 [==============================] - 0s 5ms/step - loss: 0.6715 - accuracy: 0.6096 - val_loss: 0.6548 - val_accuracy: 0.6425
Epoch 7/100
12/12 [==============================] - 0s 5ms/step - loss: 0.6700 - accuracy: 0.6096 - val_loss: 0.6524 - val_accuracy: 0.6425
Epoch 8/100
12/12 [=====

2021-10-24 20:08:01.413697: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-24 20:08:01.413733: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-24 20:08:01.414346: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


 3/12 [======>.......................] - ETA: 0s - loss: 0.6925 - accuracy: 0.5000 

2021-10-24 20:08:02.986377: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-24 20:08:02.986446: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-24 20:08:03.119586: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-10-24 20:08:03.124213: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-10-24 20:08:03.128015: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/32-nodes-14-layers-1635086281/train/plugins/profile/2021_10_24_20_08_03

2021-10-24 20:08:03.129722: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/32-nodes-14-layers-1635086281/train/plugins/profile/2021_10_24_20_08_03/root.trace.json.gz
2021-10-24 20:08:03.134009: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/32-nodes-14-layers-1

12/12 [==============================] - 2s 49ms/step - loss: 0.6885 - accuracy: 0.5758 - val_loss: 0.6751 - val_accuracy: 0.6425
Epoch 2/100
12/12 [==============================] - 0s 6ms/step - loss: 0.6749 - accuracy: 0.6096 - val_loss: 0.6569 - val_accuracy: 0.6425
Epoch 3/100
12/12 [==============================] - 0s 6ms/step - loss: 0.6694 - accuracy: 0.6096 - val_loss: 0.6499 - val_accuracy: 0.6425
Epoch 4/100
12/12 [==============================] - 0s 7ms/step - loss: 0.6681 - accuracy: 0.6096 - val_loss: 0.6573 - val_accuracy: 0.6425
Epoch 5/100
12/12 [==============================] - 0s 5ms/step - loss: 0.6702 - accuracy: 0.6096 - val_loss: 0.6571 - val_accuracy: 0.6425
Epoch 6/100
12/12 [==============================] - 0s 5ms/step - loss: 0.6678 - accuracy: 0.6096 - val_loss: 0.6494 - val_accuracy: 0.6425
Epoch 7/100
12/12 [==============================] - 0s 5ms/step - loss: 0.6654 - accuracy: 0.6096 - val_loss: 0.6436 - val_accuracy: 0.6425
Epoch 8/100
12/12 [=====

In [40]:
model = build_model(9, 12)

12-nodes-9-layers-1635087079
Epoch 1/100


2021-10-24 20:21:19.583690: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-24 20:21:19.583744: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-24 20:21:19.584684: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


 3/12 [======>.......................] - ETA: 0s - loss: 0.6931 - accuracy: 0.4896

2021-10-24 20:21:20.358231: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-24 20:21:20.358273: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-24 20:21:20.420888: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-10-24 20:21:20.422625: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-10-24 20:21:20.425500: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/12-nodes-9-layers-1635087079/train/plugins/profile/2021_10_24_20_21_20

2021-10-24 20:21:20.426390: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/12-nodes-9-layers-1635087079/train/plugins/profile/2021_10_24_20_21_20/root.trace.json.gz
2021-10-24 20:21:20.429722: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/12-nodes-9-layers-1635

12/12 [==============================] - 1s 27ms/step - loss: 0.6918 - accuracy: 0.5758 - val_loss: 0.6880 - val_accuracy: 0.6425
Epoch 2/100
12/12 [==============================] - 0s 5ms/step - loss: 0.6878 - accuracy: 0.6096 - val_loss: 0.6824 - val_accuracy: 0.6425
Epoch 3/100
12/12 [==============================] - 0s 4ms/step - loss: 0.6833 - accuracy: 0.6096 - val_loss: 0.6762 - val_accuracy: 0.6425
Epoch 4/100
12/12 [==============================] - 0s 4ms/step - loss: 0.6785 - accuracy: 0.6096 - val_loss: 0.6677 - val_accuracy: 0.6425
Epoch 5/100
12/12 [==============================] - 0s 4ms/step - loss: 0.6729 - accuracy: 0.6096 - val_loss: 0.6589 - val_accuracy: 0.6425
Epoch 6/100
12/12 [==============================] - 0s 4ms/step - loss: 0.6693 - accuracy: 0.6096 - val_loss: 0.6550 - val_accuracy: 0.6425
Epoch 7/100
12/12 [==============================] - 0s 4ms/step - loss: 0.6691 - accuracy: 0.6096 - val_loss: 0.6528 - val_accuracy: 0.6425
Epoch 8/100
12/12 [=====

In [43]:
predictions = np.squeeze(model.predict([X_test]))

predictions = np.round(predictions).astype(np.int)

In [44]:
# output = pd.DataFrame({'Sex': np.squeeze(X_val.T[:, 3]), 'Predictions': predictions, 'Actual': np.squeeze(y_val)})
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('titanic.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
